<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Data" data-toc-modified-id="Import-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Data</a></span></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Cleaning</a></span></li><li><span><a href="#Merge-data-frames" data-toc-modified-id="Merge-data-frames-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Merge data frames</a></span></li><li><span><a href="#Apply-EMD-on-data" data-toc-modified-id="Apply-EMD-on-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Apply EMD on data</a></span></li></ul></div>

- This notebook contains code for stock data preprocessing.

In [1]:
import pandas as pd
import numpy as np
import pylab as plt
from PyEMD import EMD 

## Import Data

- Data source: 
    1. 龍虎榜指數 (sent) http://data.10jqka.com.cn/market/longhu/#refCountId=db_50754c78_378
    2. 上證50指數 (sz50) http://www.cninfo.com.cn/new/index
    3. 創業板指數 (cy) http://www.cninfo.com.cn/new/index

In [2]:
# Import data
sz50 = pd.read_csv('data/stock_data/sz50.csv')
cyb = pd.read_csv('data/stock_data/cyb.csv')

sent = pd.read_csv('data/stock_data/sent_lhb.csv', header = None)
sent.columns = ['sen', 'lhb', 'date']

print(sz50.shape, cyb.shape, sent.shape)
sz50.head()

(394, 7) (394, 7) (394, 3)


,日期,收盘,开盘,高,低,交易量,涨跌幅
0,2019年9月30日,"2,897.70","2,917.07","2,933.51","2,897.68",1.92B,-1.08%
1,2019年9月27日,"2,929.47","2,929.32","2,934.33","2,916.07",2.30B,0.05%
2,2019年9月26日,"2,927.88","2,939.68","2,952.87","2,926.54",3.15B,-0.03%
3,2019年9月25日,"2,928.84","2,930.14","2,942.00","2,923.15",2.25B,-0.38%
4,2019年9月24日,"2,940.08","2,936.95","2,956.29","2,929.60",2.27B,0.31%


- Data description for sz50 and cyb
    1. Period: 20180222 - 20190930
    2. Columns: 收盤、開盤、高、低、交易量、漲跌幅

## Data Cleaning

In [3]:
# Clean date column
def clean_date(date):
    '''Convert chinese date into date'''
    date = date.split('年')  
    year = date[0]
    
    date = date[1].split('月')
    month = date[0]
    if len(month) == 1:
        month = '0' + month
    
    date = date[1].split('日')
    day = date[0]
    if len(day) == 1:
        day = '0' + day
        
    return year + month + day

sz50['date'] = sz50['日期'].apply(clean_date).astype(int)
cyb['date'] = cyb['日期'].apply(clean_date).astype(int)

sz50.drop('日期', axis=1, inplace=True)
cyb.drop('日期', axis=1, inplace=True)

sz50.head()

,收盘,开盘,高,低,交易量,涨跌幅,date
0,"2,897.70","2,917.07","2,933.51","2,897.68",1.92B,-1.08%,20190930
1,"2,929.47","2,929.32","2,934.33","2,916.07",2.30B,0.05%,20190927
2,"2,927.88","2,939.68","2,952.87","2,926.54",3.15B,-0.03%,20190926
3,"2,928.84","2,930.14","2,942.00","2,923.15",2.25B,-0.38%,20190925
4,"2,940.08","2,936.95","2,956.29","2,929.60",2.27B,0.31%,20190924


In [4]:
sz50['涨跌幅'] = sz50['涨跌幅'].str.strip('%')
cyb['涨跌幅'] = cyb['涨跌幅'].str.strip('%')

cyb['涨跌幅'] = pd.to_numeric(cyb['涨跌幅'])
sz50['涨跌幅'] = pd.to_numeric(sz50['涨跌幅'])

## Merge data frames

In [5]:
result = pd.merge(sz50, cyb, on='date', suffixes=('_sz', '_cy'))
result.head()

,收盘_sz,开盘_sz,高_sz,低_sz,交易量_sz,涨跌幅_sz,date,收盘_cy,开盘_cy,高_cy,低_cy,交易量_cy,涨跌幅_cy
0,"2,897.70","2,917.07","2,933.51","2,897.68",1.92B,-1.08,20190930,"1,627.55","1,648.90","1,651.87","1,626.05",1.68B,-1.21
1,"2,929.47","2,929.32","2,934.33","2,916.07",2.30B,0.05,20190927,"1,647.53","1,628.14","1,653.06","1,624.75",1.92B,1.46
2,"2,927.88","2,939.68","2,952.87","2,926.54",3.15B,-0.03,20190926,"1,623.79","1,677.79","1,680.93","1,623.65",2.21B,-2.92
3,"2,928.84","2,930.14","2,942.00","2,923.15",2.25B,-0.38,20190925,"1,672.65","1,686.19","1,689.23","1,670.68",2.22B,-1.30
4,"2,940.08","2,936.95","2,956.29","2,929.60",2.27B,0.31,20190924,"1,694.60","1,685.12","1,706.82","1,678.18",2.28B,0.61


In [6]:
result = pd.merge(result, sent, on='date')
result.head()

,收盘_sz,开盘_sz,高_sz,低_sz,交易量_sz,涨跌幅_sz,date,收盘_cy,开盘_cy,高_cy,低_cy,交易量_cy,涨跌幅_cy,sen,lhb
0,"2,897.70","2,917.07","2,933.51","2,897.68",1.92B,-1.08,20190930,"1,627.55","1,648.90","1,651.87","1,626.05",1.68B,-1.21,-0.049456,-0.030211
1,"2,929.47","2,929.32","2,934.33","2,916.07",2.30B,0.05,20190927,"1,647.53","1,628.14","1,653.06","1,624.75",1.92B,1.46,-0.029732,-0.036020
2,"2,927.88","2,939.68","2,952.87","2,926.54",3.15B,-0.03,20190926,"1,623.79","1,677.79","1,680.93","1,623.65",2.21B,-2.92,-0.068419,-0.098260
3,"2,928.84","2,930.14","2,942.00","2,923.15",2.25B,-0.38,20190925,"1,672.65","1,686.19","1,689.23","1,670.68",2.22B,-1.30,-0.005162,-0.041207
4,"2,940.08","2,936.95","2,956.29","2,929.60",2.27B,0.31,20190924,"1,694.60","1,685.12","1,706.82","1,678.18",2.28B,0.61,0.005044,-0.002203


In [7]:
result.sort_values(by='date', inplace=True)
result.reset_index(inplace=True, drop=True)
result.head()

,收盘_sz,开盘_sz,高_sz,低_sz,交易量_sz,涨跌幅_sz,date,收盘_cy,开盘_cy,高_cy,低_cy,交易量_cy,涨跌幅_cy,sen,lhb
0,"2,931.88","2,915.97","2,936.85","2,900.87",3.93B,2.09,20180222,"1,677.76","1,661.56","1,679.79","1,650.88",1.40B,1.88,0.016575,0.054851
1,"2,957.17","2,941.42","2,966.75","2,926.36",3.82B,0.86,20180223,"1,668.83","1,674.62","1,681.24","1,656.95",1.25B,-0.53,-0.019798,0.047175
2,"2,973.79","2,973.81","2,986.81","2,933.52",5.02B,0.56,20180226,"1,729.15","1,683.15","1,733.71","1,673.95",1.99B,3.61,0.044863,0.097175
3,"2,927.10","2,979.77","2,979.77","2,918.01",4.62B,-1.57,20180227,"1,743.54","1,729.94","1,757.37","1,728.31",2.28B,0.83,0.001990,0.037009
4,"2,878.67","2,899.00","2,905.57","2,866.46",3.88B,-1.65,20180228,"1,753.63","1,730.15","1,773.73","1,723.39",2.16B,0.58,-0.022372,-0.042866


In [8]:
result.to_csv('data/stock_data/data_sz50_cy_lhb_sen.csv')

## Apply EMD on data


In [9]:
def emd(index_col, stock_col, df_name):
    '''Apply EMD method on given stock column'''
    x = np.array(index_col)
    y = np.array(stock_col)
    IMF = EMD().emd(y,x)
    N = IMF.shape[0]+1

    temp = pd.DataFrame(data=IMF.reshape((len(IMF[0]), len(IMF))))
    temp.columns = [str(df_name) + '_' + str(i) for i in range(1, len(IMF)+1)]
    
    return temp

In [10]:
# Apply EMD on sz50 and cyb
emd_涨跌幅_sz = emd(result.index, result['涨跌幅_sz'], 'emd_涨跌幅_sz')
emd_涨跌幅_cy = emd(result.index, result['涨跌幅_cy'], 'emd_涨跌幅_cy')

emd_涨跌幅_cy.to_csv('data/stock_data/emd_涨跌幅_cy.csv')
emd_涨跌幅_sz.to_csv('data/stock_data/emd_涨跌幅_sz.csv')